In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [4]:
!pip install geopy

     |████████████████████████████████| 112kB 107kB/s eta 0:00:01


#### Scraping the wikipedia page for postal codes of Canada

In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

In [4]:
tables = soup.find_all('table')

In [6]:
req_table=tables[0]

In [7]:
for i in req_table:
    rows2=req_table.find_all('td')

In [8]:
len(rows2)

180

In [9]:
pincode=[]
burough=[]
neighbourhood=[]
for i in range(0,len(rows2)):
    if(len(rows2[i].findAll('a'))==1):
        pincode.append(rows2[i].findAll('b')[0].text)
        burough.append(rows2[i].findAll('a')[0].get('title'))
        #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
        neighbourhood.append(rows2[i].findAll('a')[0].get('title'))
    elif(len(rows2[i].findAll('a'))>=2):
        for j in range(1,len(rows2[i].findAll('a'))):
            pincode.append(rows2[i].findAll('b')[0].text)
            burough.append(rows2[i].findAll('a')[0].get('title'))
            neighbourhood.append(rows2[i].findAll('a')[j].get('title'))
    else:
        pincode.append(rows2[i].findAll('b')[0].text)
        burough.append('0')
        neighbourhood.append('0')

In [10]:
type(pincode)

list

In [11]:
len(pincode)

253

In [12]:
len(burough)

253

In [13]:
len(neighbourhood)

253

In [14]:
import pandas as pd
df_final = pd.DataFrame(
    {'PostalCode': pincode,
     'Borough': burough,
     'Neighborhood': neighbourhood
    })


####Only processing the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [15]:
df_fin=df_final[df_final['Borough']!='0']

In [16]:
df2=df_fin.groupby("PostalCode").count()['Neighborhood']>1

In [17]:
df2=df2.reset_index()

In [18]:
list_pincode_single=df2[df2['Neighborhood']==False]['PostalCode'].tolist()

In [19]:
list_pincode_multiple=df2[df2['Neighborhood']==True]['PostalCode'].tolist()

In [20]:
cols={'PostalCode','Borough','Neighborhood'}
k=pd.DataFrame(columns=cols)
for i in range(0,len(df_fin)):
    if(df_fin.iloc[i]['PostalCode'] in list_pincode_single):
        k=k.append(df_fin.iloc[i])  

In [21]:
len(list_pincode_multiple)

44

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [22]:
pincode2=[]
burough2=[]
neighbourhood2=[]

In [23]:
for j in range(0,len(list_pincode_multiple)):
        pincode2.append(list_pincode_multiple[j])
        #print(pincode2)
        burough2.append(df_fin[df_fin['PostalCode']==list_pincode_multiple[j]]['Borough'].iloc[0])
        str1=""
        neighbourhood2.append(str1.join(df_fin[df_fin['PostalCode']==list_pincode_multiple[j]]['Neighborhood'].tolist()))  

In [24]:
len(pincode2)

44

In [25]:
len(burough2)

44

In [26]:
len(neighbourhood2)

44

In [27]:
import pandas as pd
df_multipleeq = pd.DataFrame({'PostalCode': pincode2,'Borough': burough2,'Neighborhood': neighbourhood2})


In [28]:
len(k)

57

In [29]:
final=df_multipleeq.append(k)

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [30]:
final.shape

(101, 3)

In [32]:
final.head()

,PostalCode,Borough,Neighborhood
0,M1B,"Scarborough, Toronto","Malvern, TorontoRouge, Toronto"
1,M1C,"Scarborough, Toronto","Rouge HillPort Union, OntarioHighland Creek, T..."
2,M1E,"Scarborough, Toronto","GuildwoodMorningside, TorontoWest Hill, Toronto"
3,M1K,"Scarborough, Toronto","Kennedy Park, TorontoIonviewBirchmount Park"
4,M1L,"Scarborough, Toronto","Golden Mile, TorontoClairleaOakridge, Toronto"


In [505]:
!pip install geocoder

     |████████████████████████████████| 102kB 115kB/s ta 0:00:01


### Getting the latitudes and longitudes using geocoder

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
postal_code='M5G'
# loop until you get the coordinates
while(lat_lng_coords is None):
    g= geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [510]:
pwd

'/Users/attreys'

### Getting latitudes and longitudes from the csv file

In [34]:
df=pd.read_csv('Geospatial_Coordinates.csv')

In [35]:
df_withlong=final.merge(df,left_on='PostalCode', right_on='Postal Code')

In [39]:
df_withlong=df_withlong.drop(columns=['Postal Code'])

In [40]:
df_withlong.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,"Scarborough, Toronto","Malvern, TorontoRouge, Toronto",43.806686,-79.194353
1,M1C,"Scarborough, Toronto","Rouge HillPort Union, OntarioHighland Creek, T...",43.784535,-79.160497
2,M1E,"Scarborough, Toronto","GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711
3,M1K,"Scarborough, Toronto","Kennedy Park, TorontoIonviewBirchmount Park",43.727929,-79.262029
4,M1L,"Scarborough, Toronto","Golden Mile, TorontoClairleaOakridge, Toronto",43.711112,-79.284577


### Get boroughs that contain the word Toronto

In [41]:
df_toronto=df_withlong[df_withlong['Borough'].str.contains("Toronto")]
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,"Scarborough, Toronto","Malvern, TorontoRouge, Toronto",43.806686,-79.194353
1,M1C,"Scarborough, Toronto","Rouge HillPort Union, OntarioHighland Creek, T...",43.784535,-79.160497
2,M1E,"Scarborough, Toronto","GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711
3,M1K,"Scarborough, Toronto","Kennedy Park, TorontoIonviewBirchmount Park",43.727929,-79.262029
4,M1L,"Scarborough, Toronto","Golden Mile, TorontoClairleaOakridge, Toronto",43.711112,-79.284577


### Foursquare API connection

In [42]:
CLIENT_ID = 'WWBWY31GGFBBNMQNIFQ3C43LBNZGQFLR11YVTRPVVCM14YXQ' # your Foursquare ID
CLIENT_SECRET = 'AVAVKSAMVGH3KCBHNLMOX5M444X4VZIF5TQ035QAC0TC424V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WWBWY31GGFBBNMQNIFQ3C43LBNZGQFLR11YVTRPVVCM14YXQ
CLIENT_SECRET:AVAVKSAMVGH3KCBHNLMOX5M444X4VZIF5TQ035QAC0TC424V


In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    LIMIT=100
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [44]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Malvern, TorontoRouge, Toronto
Rouge HillPort Union, OntarioHighland Creek, Toronto
GuildwoodMorningside, TorontoWest Hill, Toronto
Kennedy Park, TorontoIonviewBirchmount Park
Golden Mile, TorontoClairleaOakridge, Toronto
Cliffside, TorontoCliffcrestScarborough Village
Birch CliffCliffside, Toronto
Dorset ParkWexford HeightsScarborough Town Centre
Wexford, TorontoMaryvale, Toronto
Tam O'Shanter, OntarioSullivan, Toronto
Milliken, TorontoAgincourt NorthSteeles, TorontoL'Amoreaux
Steeles, TorontoL'Amoreaux
RathnellySouth Hill, TorontoForest Hill, TorontoDeer Park, Toronto
St. James TownCabbagetown (Toronto)
Regent ParkHarbourfront, Toronto
Garden District, TorontoRyerson University
Richmond StreetKing Street (Toronto)
Harbourfront (Toronto)Union Station (Toronto)Toronto Islands
Toronto Dominion CentreDesign Exchange
Commerce CourtHotel Victoria (Toronto)
Kensington MarketChinatown, TorontoGrange Park (Toronto)
CN TowerKing and SpadinaRailway LandsHarbourfront (Toronto)South NiagaraBilly 

In [45]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, TorontoRouge, Toronto",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge HillPort Union, OntarioHighland Creek, T...",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,"GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [46]:
print(toronto_venues.shape)
toronto_venues.head()

(1590, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, TorontoRouge, Toronto",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge HillPort Union, OntarioHighland Creek, T...",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,"GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


#### Count the number of venues per neighbourhood

In [47]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Agincourt, Toronto",4,4,4,4,4,4
Bay Street,78,78,78,78,78,78
"Birch CliffCliffside, Toronto",4,4,4,4,4,4
CN TowerKing and SpadinaRailway LandsHarbourfront (Toronto)South NiagaraBilly Bishop Toronto City Airport,17,17,17,17,17,17
Caledonia-Fairbanks,4,4,4,4,4,4
Church and Wellesley,83,83,83,83,83,83
"Cliffside, TorontoCliffcrestScarborough Village",3,3,3,3,3,3
Commerce CourtHotel Victoria (Toronto),100,100,100,100,100,100
Dorset ParkWexford HeightsScarborough Town Centre,5,5,5,5,5,5


#### Let's find out how many unique categories can be curated from all the returned venues

In [48]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


In [49]:
####Analyze each neighbourhood

In [50]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Malvern, TorontoRouge, Toronto",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [51]:
toronto_onehot.shape

(1590, 231)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [52]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Agincourt, Toronto",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

#### Let's print each neighborhood along with the top 5 most common venues

In [53]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt, Toronto----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2             Clothing Store  0.25
3  Latin American Restaurant  0.25
4                Yoga Studio  0.00


----Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.05
2        Burger Joint  0.04
3      Sandwich Place  0.04
4     Thai Restaurant  0.04


----Birch CliffCliffside, Toronto----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4                  Motel  0.00


----CN TowerKing and SpadinaRailway LandsHarbourfront (Toronto)South NiagaraBilly Bishop Toronto City Airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4     Boat or Ferry  0.06


----Caledonia-Fairbanks----
                             venue  freq
0    

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agincourt, Toronto",Latin American Restaurant,Clothing Store,Breakfast Spot,Lounge,Women's Store,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Thai Restaurant,Burger Joint,Bubble Tea Shop,Ice Cream Shop,Middle Eastern Restaurant,Café
2,"Birch CliffCliffside, Toronto",Café,College Stadium,Skating Rink,General Entertainment,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
3,CN TowerKing and SpadinaRailway LandsHarbourfr...,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Plane,Sculpture Garden
4,Caledonia-Fairbanks,Park,Market,Women's Store,Greek Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [57]:
### Cluster neighbourhoods

In [59]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Scarborough, Toronto","Malvern, TorontoRouge, Toronto",43.806686,-79.194353,3.0,Fast Food Restaurant,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M1C,"Scarborough, Toronto","Rouge HillPort Union, OntarioHighland Creek, T...",43.784535,-79.160497,2.0,Bar,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,M1E,"Scarborough, Toronto","GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,1.0,Electronics Store,Rental Car Location,Spa,Breakfast Spot,Medical Center,Bank,Intersection,Mexican Restaurant,Eastern European Restaurant,Dumpling Restaurant
3,M1K,"Scarborough, Toronto","Kennedy Park, TorontoIonviewBirchmount Park",43.727929,-79.262029,1.0,Coffee Shop,Hobby Shop,Discount Store,Bus Station,Department Store,Women's Store,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant
4,M1L,"Scarborough, Toronto","Golden Mile, TorontoClairleaOakridge, Toronto",43.711112,-79.284577,1.0,Bakery,Bus Line,Intersection,Metro Station,Park,Ice Cream Shop,Soccer Field,Bus Station,Discount Store,Women's Store


###visualize the clusters

In [61]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [62]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [63]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Scarborough, Toronto","Malvern, TorontoRouge, Toronto",43.806686,-79.194353,3.0,Fast Food Restaurant,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M1C,"Scarborough, Toronto","Rouge HillPort Union, OntarioHighland Creek, T...",43.784535,-79.160497,2.0,Bar,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,M1E,"Scarborough, Toronto","GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,1.0,Electronics Store,Rental Car Location,Spa,Breakfast Spot,Medical Center,Bank,Intersection,Mexican Restaurant,Eastern European Restaurant,Dumpling Restaurant
3,M1K,"Scarborough, Toronto","Kennedy Park, TorontoIonviewBirchmount Park",43.727929,-79.262029,1.0,Coffee Shop,Hobby Shop,Discount Store,Bus Station,Department Store,Women's Store,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant
4,M1L,"Scarborough, Toronto","Golden Mile, TorontoClairleaOakridge, Toronto",43.711112,-79.284577,1.0,Bakery,Bus Line,Intersection,Metro Station,Park,Ice Cream Shop,Soccer Field,Bus Station,Discount Store,Women's Store


In [70]:
rainbow = [colors.rgb2hex(i) for i in colors_array]
type(rainbow)

list

In [73]:
toronto_merged['Cluster Labels']

0     3.0
1     2.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    0.0
11    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
27    1.0
28    1.0
29    1.0
31    1.0
32    1.0
33    1.0
34    1.0
50    1.0
51    1.0
54    1.0
55    0.0
56    1.0
58    1.0
59    1.0
60    1.0
64    4.0
67    1.0
72    1.0
79    0.0
80    0.0
83    0.0
87    1.0
88    1.0
91    1.0
93    1.0
94    1.0
95    4.0
96    0.0
97    1.0
98    NaN
99    1.0
Name: Cluster Labels, dtype: float64

In [80]:
toronto_merged=toronto_merged.dropna()
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Scarborough, Toronto","Malvern, TorontoRouge, Toronto",43.806686,-79.194353,3.0,Fast Food Restaurant,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M1C,"Scarborough, Toronto","Rouge HillPort Union, OntarioHighland Creek, T...",43.784535,-79.160497,2.0,Bar,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,M1E,"Scarborough, Toronto","GuildwoodMorningside, TorontoWest Hill, Toronto",43.763573,-79.188711,1.0,Electronics Store,Rental Car Location,Spa,Breakfast Spot,Medical Center,Bank,Intersection,Mexican Restaurant,Eastern European Restaurant,Dumpling Restaurant
3,M1K,"Scarborough, Toronto","Kennedy Park, TorontoIonviewBirchmount Park",43.727929,-79.262029,1.0,Coffee Shop,Hobby Shop,Discount Store,Bus Station,Department Store,Women's Store,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant
4,M1L,"Scarborough, Toronto","Golden Mile, TorontoClairleaOakridge, Toronto",43.711112,-79.284577,1.0,Bakery,Bus Line,Intersection,Metro Station,Park,Ice Cream Shop,Soccer Field,Bus Station,Discount Store,Women's Store


In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        in_num=int(cluster-1),
        color=rainbow[in_num],
        fill=True,
        fill_color=rainbow[in_num],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

### Cluster1

In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]



,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Scarborough, Toronto",0.0,Playground,Park,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
55,"York, Toronto",0.0,Park,Market,Women's Store,Greek Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
79,"Lawrence Park, Toronto",0.0,Park,Swim School,Bus Line,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
80,"York, Toronto",0.0,Park,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
83,"Forest Hill, Toronto",0.0,Park,Jewelry Store,Trail,Sushi Restaurant,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
96,Downtown Toronto,0.0,Park,Playground,Trail,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


### Cluster 2

In [93]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Scarborough, Toronto",1.0,Electronics Store,Rental Car Location,Spa,Breakfast Spot,Medical Center,Bank,Intersection,Mexican Restaurant,Eastern European Restaurant,Dumpling Restaurant
3,"Scarborough, Toronto",1.0,Coffee Shop,Hobby Shop,Discount Store,Bus Station,Department Store,Women's Store,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant
4,"Scarborough, Toronto",1.0,Bakery,Bus Line,Intersection,Metro Station,Park,Ice Cream Shop,Soccer Field,Bus Station,Discount Store,Women's Store
5,"Scarborough, Toronto",1.0,American Restaurant,Movie Theater,Motel,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
6,"Scarborough, Toronto",1.0,Café,College Stadium,Skating Rink,General Entertainment,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
7,"Scarborough, Toronto",1.0,Indian Restaurant,Chinese Restaurant,Pet Store,Vietnamese Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Women's Store
8,"Scarborough, Toronto",1.0,Shopping Mall,Middle Eastern Restaurant,Vietnamese Restaurant,Breakfast Spot,Sandwich Place,Auto Garage,Bakery,Dog Run,Discount Store,Distribution Center
9,"Scarborough, Toronto",1.0,Pizza Place,Bank,Fast Food Restaurant,Noodle House,Shopping Mall,Italian Restaurant,Rental Car Location,Pharmacy,Gas Station,Intersection
11,"Scarborough, Toronto",1.0,Chinese Restaurant,Fast Food Restaurant,Gym,Pizza Place,Bank,Electronics Store,Bubble Tea Shop,Cosmetics Shop,Supermarket,Thrift / Vintage Store
17,"Summerhill, Toronto",1.0,Coffee Shop,Pub,Burger Joint,American Restaurant,Sushi Restaurant,Light Rail Station,Bank,Sports Bar,Fried Chicken Joint,Pizza Place


### Cluster3

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Scarborough, Toronto",2.0,Bar,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster 4

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Scarborough, Toronto",3.0,Fast Food Restaurant,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster 5

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,"Scarborough, Toronto",4.0,Playground,Convenience Store,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
95,"Moore Park, Toronto",4.0,Playground,Women's Store,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
